In [1]:
 !pip install transformers

# 1) KoGPT-2로 문장 생성하기


In [2]:
import numpy as np
import random
import tensorflow as tf
from transformers import AutoTokenizer
from transformers import TFGPT2LMHeadModel

In [3]:
model = TFGPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2', from_pt = True)
tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/513M [00:00<?, ?B/s]

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2LMHeadModel: ['transformer.h.9.attn.masked_bias', 'lm_head.weight', 'transformer.h.2.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.6.attn.masked_bias']
- This IS expected if you are initializing TFGPT2LMHeadModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT

tokenizer.json: 0.00B [00:00, ?B/s]

In [4]:
sent = '근육이 커지기 위해서는'

In [7]:
#정수 시퀀스로 변환
input_id = tokenizer.encode(sent)
input_ids = tf.convert_to_tensor([input_id])
print(input_id)
print(input_ids)

[33245, 10114, 12748, 11357]
tf.Tensor([[33245 10114 12748 11357]], shape=(1, 4), dtype=int32)


In [8]:
#주어진 문장으로부터 이어진 문장 생성
output = model.generate(input_ids, max_length = 128, repetition_penalty =2.0, use_cache = True)
output_ids = output.numpy().tolist()[0]
print(output_ids)

[33245, 10114, 12748, 11357, 23879, 39306, 9684, 7884, 10211, 15177, 26421, 387, 17339, 7889, 9908, 15768, 6903, 15386, 8146, 12923, 9228, 18651, 42600, 9564, 17764, 9033, 9199, 14441, 7335, 8704, 12557, 32030, 9510, 18595, 9025, 10571, 25741, 10599, 13229, 9508, 7965, 8425, 33102, 9122, 21240, 9801, 32106, 13579, 12442, 13235, 19430, 8022, 12972, 9566, 11178, 9554, 24873, 7198, 9391, 12486, 8711, 9346, 7071, 36736, 9693, 12006, 9038, 10279, 36122, 9960, 8405, 10826, 18988, 25998, 9292, 7671, 9465, 7489, 9277, 10137, 9677, 9248, 9912, 12834, 11488, 13417, 7407, 8428, 8137, 9430, 14222, 11356, 10061, 9885, 19265, 9377, 20305, 7991, 9178, 9648, 9133, 10021, 10138, 30315, 21833, 9362, 9301, 9685, 11584, 9447, 42129, 10124, 7532, 17932, 47123, 37544, 9355, 15632, 9124, 10536, 13530, 12204, 9184, 36152, 9673, 9788, 9029, 11764]


In [9]:
tokenizer.decode(output_ids)

'근육이 커지기 위해서는 무엇보다 규칙적인 생활습관이 중요하다.\n특히, 아침식사는 단백질과 비타민이 풍부한 과일과 채소를 많이 섭취하는 것이 좋다.\n또한 하루 30분 이상 충분한 수면을 취하는 것도 도움이 된다.\n아침 식사를 거르지 않고 규칙적으로 운동을 하면 혈액순환에 도움을 줄 뿐만 아니라 신진대사를 촉진해 체내 노폐물을 배출하고 혈압을 낮춰준다.\n운동은 하루에 10분 정도만 하는 게 좋으며 운동 후에는 반드시 스트레칭을 통해 근육량을 늘리고 유연성을 높여야 한다.\n운동 후 바로 잠자리에 드는 것은 피해야 하며 특히 아침에 일어나면 몸이 피곤해지기 때문에 무리하게 움직이면 오히려 역효과가 날 수도 있다.\n운동을'

# 2) Numpy로 Top 5 뽑기

In [12]:
output = model(input_ids)
top5 = tf.math.top_k(output.logits[0,-1], k =5)

In [13]:
tokenizer.convert_ids_to_tokens(top5.indices.numpy())

['▁무엇보다', '▁우선', '▁반드시', '▁피부', '▁무엇보다도']

# 3) Top5로 문장 생성하기

In [14]:
sent = '근육이 커지기 위해서는'
input_ids = tokenizer.encode(sent)

while len(input_ids) < 50:
    output = model(np.array([input_ids]))
    top5 = tf.math.top_k(output.logits[0,-1], k =5)
    token_id = random.choice(top5.indices.numpy()) #top5 단어 중 랜덤으로 단어 선택
    input_ids.append(token_id)

tokenizer.decode(input_ids)

'근육이 커지기 위해서는 우선 피부 타입을 정확히 파악한 후 적절한 보톡스로 피부의 볼륨감과 리프팅을 동시에 해결하고 피부의 처진 부분을 채워주는 게 좋다.\n보철물을 사용하는 것이 좋으며 보습과 보습을 위해 사용하는 것이 가장 효과적이다.\n또'